# I. Setting up the Problem

In [2]:
import pandas as pd
import numpy as np

# Import the random forest package
from sklearn.ensemble import RandomForestClassifier 

In [3]:
filename ="CrowdstormingDataJuly1st.csv"
Data = pd.read_csv(filename)

### 1) Peeking into the Data

In [4]:
Data.ix[:10,:13]

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,ties,defeats,goals
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,0,1,0
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,0,1,0
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,1,0,0
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,0,0,0
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,0,0,0
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,0,1,0
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,0,0,0
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,0,1,0
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,1,0,0
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,0,0,0


In [5]:
#Data.ix[:10,13:28]

# II. Preparing the training & test data : Unique Game Row version

### 1) Keep only players that have a Rater Image

In [6]:
# 1) Remove the players without rater 1 / 2 rating because we won't be 
# able to train or test the values (this can be done as bonus later)

Data_hasImage = Data[pd.notnull(Data['photoID'])]
#Data_hasImage.ix[:10,13:28]

### 2) Disaggregate the data so each row is 1 game

Got a lot of help from this script ! https://osf.io/w7tds/
It will be much simpler for us to train our random forest if each row corresponds to one game. This way, we won't have to give a different "weight" to each row according to the number of played games.

But let's start by doing the mean value of rater1 and rater 2, because if we keep them separated we might get some strange results.
Indeed, what if for a player, rater1 = 0.0 and rater2 = 0.75 ?
It would not make a lot of sense, or at least we would know our model is not viable !

In [7]:
Data_hasImage['mean_rater']=(Data_hasImage['rater1']+Data_hasImage['rater2'])/2

C:\Users\merin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


Let's now disaggregate the games:

In [8]:
game_counter = 0
game_total_number = sum(Data_hasImage['games'])
# Raw table that we'll have to convert to a dataframe later
output = [0 for i in range(game_total_number)]

# We now iterate each row of our dataframe, which may contains more that one game
for i, row in Data_hasImage.iterrows():
    # Number of games in the current row
    row_game_number = row['games']
    # Number of cumulated cards for the games in the current row
    yellowCards = row['yellowCards']
    yellowReds = row['yellowReds']
    redCards = row['redCards']
    # We want to seperate each of these games    
    for j in range (row_game_number):
        game = row
        game['yellowCards'] = 0
        game['yellowReds'] = 0
        game['redCards'] = 0
        # Basically, we distribute the cards we have on separate games.
        # ie: if we have 2 yellowCard and 1 redCard for a total of 4 games,
        # the first two games will be assigned a yellowCard,
        # the third game will be assigned a redCard,
        # and the last game won't have any card assigned, because there is no card left.        
        if yellowCards > 0:
            game['yellowCards'] = 1
            yellowCards = yellowCards - 1
        elif yellowReds > 0:
            game['yellowReds'] = 1
            yellowReds = yellowReds - 1
        elif redCards > 0:
            game['redCards'] = 1
            redCards = redCards - 1
            
        # Convert from pandas Series to prevent overwriting previous values of the output
        gamelist=list(game)
        # Add the new game to the output
        output[game_counter] = gamelist
        game_counter = game_counter + 1

# Here is the output dataframe

Data_OneGamePerRow = pd.DataFrame(output, columns=list(Data_hasImage.columns))
Data_OneGamePerRow

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,mean_rater
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696,0.375
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504,0.750
2,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.125
3,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.125
4,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,1.000
5,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.250
6,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.000
7,antonio-rukavina,Antonio Rukavina,Real Valladolid,Spain,26.01.1984,177.0,74.0,Right Fullback,2,2,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.000
8,antonio-rukavina,Antonio Rukavina,Real Valladolid,Spain,26.01.1984,177.0,74.0,Right Fullback,2,2,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.000
9,ashkan-dejagah,Ashkan Dejagah,Fulham FC,England,05.07.1986,181.0,74.0,Left Winger,1,1,...,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.500


### 3) Create the Training and Testing Datframes with only select data

In [9]:
# Removing columns that we do not need
Data_Simple1 = Data_hasImage[['playerShort', 'games', 'yellowCards', 'yellowReds', 'redCards',
                              'refNum', 'refCountry', 'mean_rater']]

# Take a random 80% sample of the Data for the Training Sample
#Data_Training = Data_Simple1.sample(frac=0.8)

# Take a random 20% sample of the Data for the Testing Sample
#Data_Testing = Data_Simple1.loc[~Data_Simple1.index.isin(Data_Training.index)]

In [10]:
Data_Simple1

,playerShort,games,yellowCards,yellowReds,redCards,refNum,refCountry,mean_rater
0,lucas-wilchez,1,0,0,0,1,1,0.375
1,john-utaka,1,1,0,0,2,2,0.750
5,aaron-hughes,1,0,0,0,4,4,0.125
6,aleksandar-kolarov,1,0,0,0,4,4,0.125
7,alexander-tettey,1,0,0,0,4,4,1.000
8,anders-lindegaard,1,0,0,0,4,4,0.250
9,andreas-beck,1,0,0,0,4,4,0.000
10,antonio-rukavina,2,1,0,0,4,4,0.000
11,ashkan-dejagah,1,0,0,0,4,4,0.500
12,benedikt-hoewedes,1,0,0,0,4,4,0.000


We need to aggregate the information about referees and group the result by soccer player. It means that each line will correspond to a soccer player, with the sum of all the cards he got, and we won't know anymore who gaves the cards.

In [13]:
# Group by player and do the sum of every column, except the numberof game and mean_rater that we need to move away during the calculation
Data_aggregated = Data_hasImage.drop(['refNum', 'refCountry'], 1)
Data_aggregated = Data_aggregated.groupby('playerShort')['games','yellowCards', 'yellowReds', 'redCards'].sum()
# Put again the information about skin color (mean_rater)
#Data_aggregated.set_index('playerShort')
#Data_aggregated['skin_color'] = Data_hasImage['mean_rater']
#Data_aggregated['games'] = Data_hasImage['games']
Data_aggregated = Data_aggregated.reset_index()
#Data_aggregated = Data_aggregated.set_index('playerShort')
#Data_hasImage_indexedByPlayer = Data_hasImage.set_index('playerShort')
#Data_aggregated['skinColor'] = Data_hasImage_indexedByPlayer['mean_rater']
#Data_aggregated
#Data_hasImage_indexedByPlayer
#Data_aggregated
Data_aggregated

,playerShort,games,yellowCards,yellowReds,redCards
0,aaron-hughes,654,19,0,0
1,aaron-hunt,336,42,0,1
2,aaron-lennon,412,11,0,0
3,aaron-ramsey,260,31,0,1
4,abdelhamid-el-kaoutari,124,8,4,2
5,abdou-traore_2,97,11,1,0
6,abdoulaye-diallo_2,24,0,0,0
7,abdoulaye-keita_2,3,0,0,0
8,abdoulwhaid-sissoko,121,21,0,2
9,abdul-rahman-baba,50,3,0,1


In [12]:
Data_hasImage[Data_hasImage['playerShort'] == 'aaron-lennon']

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,mean_rater
746,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,2,1,...,66,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.25
1919,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,9,6,...,77,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.25
2303,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,2,2,...,88,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.25
5033,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,1,0,...,131,27,ROU,0.385082,537.0,0.000756,0.859431,562.0,0.003620,0.25
7989,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,21,8,...,194,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.25
9964,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,1,1,...,235,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.25
11000,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,4,3,...,256,44,ENGL,0.326690,44791.0,0.000010,0.356446,46916.0,0.000037,0.25
12229,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,1,1,...,274,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002,0.25
13269,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,1,1,...,282,7,FRA,0.334684,2882.0,0.000151,0.336101,3011.0,0.000586,0.25
13951,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,1,1,...,284,80,FIN,0.396512,2273.0,0.000187,1.031407,2388.0,0.000931,0.25


,playerShort,mean_rater
0,lucas-wilchez,0.375
1,john-utaka,0.750
5,aaron-hughes,0.125
6,aleksandar-kolarov,0.125
7,alexander-tettey,1.000
8,anders-lindegaard,0.250
9,andreas-beck,0.000
10,antonio-rukavina,0.000
11,ashkan-dejagah,0.500
12,benedikt-hoewedes,0.000


In [14]:
# Take information of number of games and skin color for each player
Data_nbGames_skinColor = Data_hasImage.drop_duplicates(subset='playerShort')
Data_aggregated['skinColor'] = Data_nbGames_skinColor['mean_rater']
#Data_aggregated['numberOfGames'] = Data_nbGames_skinColor['games']
Data_aggregated

,playerShort,games,yellowCards,yellowReds,redCards,skinColor
0,aaron-hughes,654,19,0,0,0.375
1,aaron-hunt,336,42,0,1,0.750
2,aaron-lennon,412,11,0,0,NaN
3,aaron-ramsey,260,31,0,1,NaN
4,abdelhamid-el-kaoutari,124,8,4,2,NaN
5,abdou-traore_2,97,11,1,0,0.125
6,abdoulaye-diallo_2,24,0,0,0,0.125
7,abdoulaye-keita_2,3,0,0,0,1.000
8,abdoulwhaid-sissoko,121,21,0,2,0.250
9,abdul-rahman-baba,50,3,0,1,0.000


In [16]:
Data_nbGames_skinColor[Data_nbGames_skinColor['playerShort'] == 'aaron-lennon']

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp,mean_rater
746,aaron-lennon,Aaron Lennon,Tottenham Hotspur,England,16.04.1987,165.0,63.0,Right Midfielder,2,1,...,66,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752,0.25


# III. Random Forest

In [110]:
#Initialize
forest = RandomForestClassifier(n_estimators = 100)

# Fit the training data and create the decision trees
forest.fit(trainArr,trainRes)

# Take the same decision trees and run it on the test data
#testArr = test.as_matrix(cols)
#results = rf.predict(testArr)

#test['predictions'] = results
#test.head()

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)